# Prepare RDF for SKOSMOS

SKOSMOS has a simple, plain-SKOS [data model](https://github.com/NatLibFi/Skosmos/wiki/Data-Model). Our data model derives from this in two major ways that break SKOSMOS’ expectations:

1. We use SKOS-XL as the lexicalization scheme, and
2. we use the more specific sub-properties of `skos:broader` defined by skos-thes.

We map these to plain SKOS before loading the dataset into SKOSMOS.

In [1]:
import rdflib
from rdflib.namespace import SKOS

skosxl = rdflib.Namespace('http://www.w3.org/2008/05/skos-xl#')
skos_thes = rdflib.Namespace('http://purl.org/iso25964/skos-thes#')

In [2]:
g = rdflib.Graph()
g.parse('diga_terms_vocbench.ttl')

<Graph identifier=N3843b06476f94ac6b32d6ff869360e18 (<class 'rdflib.graph.Graph'>)>

## Broaders

We simply map the more specific broader properties onto `skos:broader`.

In [3]:
qres = g.query(
    """SELECT ?part ?parent
       WHERE {
          ?part skos-thes:broaderPartitive ?parent .
       } LIMIT 10""")

for part, parent in qres:
    print(part, parent)

https://w3id.org/diga/terms/100088298 https://w3id.org/diga/terms/2994146436
https://w3id.org/diga/terms/1460785757 https://w3id.org/diga/terms/2994146436
https://w3id.org/diga/terms/3163172208 https://w3id.org/diga/terms/2994146436
https://w3id.org/diga/terms/1001583333 https://w3id.org/diga/terms/3410718195
https://w3id.org/diga/terms/1226134970 https://w3id.org/diga/terms/3410718195
https://w3id.org/diga/terms/1821530743 https://w3id.org/diga/terms/3410718195
https://w3id.org/diga/terms/3056398972 https://w3id.org/diga/terms/3410718195
https://w3id.org/diga/terms/4196734874 https://w3id.org/diga/terms/3410718195
https://w3id.org/diga/terms/596669033 https://w3id.org/diga/terms/3410718195
https://w3id.org/diga/terms/938843014 https://w3id.org/diga/terms/3410718195


In [4]:
g.update(
    """INSERT { ?part skos:broader ?parent . }
       WHERE {
          ?part skos-thes:broaderPartitive ?parent .
       }""")

In [5]:
g.update(
    """INSERT { ?facet skos:broader ?parent . }
       WHERE {
          ?facet skos-thes:broaderGeneric ?parent .
       }""")

## Lexicalization

We add plain skos labels based on the SKOS-XL properties.

In [6]:
qres = g.query(
    """SELECT ?concept ?label
       WHERE {
           ?concept skosxl:prefLabel ?xlabel .
           ?xlabel skosxl:literalForm ?label .
       } LIMIT 10""")

for c, l in qres:
    print(c, l)

https://w3id.org/diga/terms/1000747715 volti in alto e in basso, a foglie lanceolate e boccioli di fiori
https://w3id.org/diga/terms/1000747715 spreading upwards and downwards, with lanceolate leaves and blossoms
https://w3id.org/diga/terms/100088298 orlo
https://w3id.org/diga/terms/100088298 edge
https://w3id.org/diga/terms/1001583333 semiarchetto laterale (dx./ sn.)
https://w3id.org/diga/terms/1001583333 side small half arch (right/ left)
https://w3id.org/diga/terms/1001756703 torus
https://w3id.org/diga/terms/1001756703 toro
https://w3id.org/diga/terms/100479688 piramide a gradini
https://w3id.org/diga/terms/100479688 stepped pyramid


In [7]:
g.update(
    """INSERT { ?concept skos:prefLabel ?label . }
       WHERE {
           ?concept skosxl:prefLabel ?xlabel .
           ?xlabel skosxl:literalForm ?label .
       }""")

In [8]:
g.update(
    """INSERT { ?concept skos:altLabel ?label . }
       WHERE {
           ?concept skosxl:altLabel ?xlabel .
           ?xlabel skosxl:literalForm ?label .
       }""")

## Save

In [9]:
with open('diga_terms_skosmos.ttl', 'wb') as outfile:
    g.serialize(destination=outfile, format='turtle')